In [103]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import numpy as np
from numpy.linalg import norm
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories1 import *

In [104]:
asp_file = os.path.join("projects","minotaur","canonical_minotaur1.lp")
generated = os.path.join("generated_solutions","minotaur_from_seeknumbers_and_creek_v1.lp")
instance = os.path.join("projects","minotaur","instances","level01.lp")

In [107]:
def asp_try_3(asp_file,instance):
    errors = []
    messages = []
    error_dict = {}
    message_dict = {}
    def custom_logger(code, message):
        errors.append(code)
        messages.append(message)
    control = clingo.Control(logger=custom_logger,message_limit=100)
    input_files = [asp_file, instance]
    asp_program = []
    models = []
    symbols = []
    
    def on_model(model):
        print("Model:", model)
        with open('output.txt','w') as file:
            print(model, file=file)
        with open('output.txt','r') as file:
            lines = file.readlines()
            symbols_list = lines[0].split()
            for s in symbols_list:
                symbols.append(s)
        os.remove("output.txt") 
        #print(type(model))
        #symbols = model.symbols(atoms=False,terms=True, shown=True)
        #for s in symbols:
        #    print(s.arguments)
        #    print(type(s))
        #    symbols_list.append(s)
        #print("Symbols:", symbols)
        
         #def symbols(self, atoms: bool = False, terms: bool = False, 
         #            shown: bool = False, theory: bool = False, complement: bool = False) ‑> Sequence[Symbol] 
        models.append(model)
    
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
    control.add("base", [], "".join(asp_program))
    try:
        control.ground([("base", [])])
    except:
        print("Grounding stopped because of errors")
        error_codes = ["AtomUndefined","FileIncluded","GlobalVariable","OperationUndefined","Other","RuntimeError","VariableUnbounded"]
        sub_codes = ["unsafe"]
        
        for error_code in error_codes:
            error_dict[error_code] = 0
        
        for sub_code in sub_codes:
            sub_dict[sub_code] = 0
        for e in errors:
            for ec in error_codes:
                if str(e).split(".")[1] == ec:
                    error_dict[ec] += 1
        for mss in messages:
            for sc in sub_codes:
                if sc in mss:
                    sub_dict[sc] += 1
        return models, error_dict, message_dict
        
    control.configuration.solve.models = 0  # Limit the number of models to 1
    control.solve(on_model=on_model)
    
    return models, error_dict, message_dict, symbols

In [108]:
models, error_dict, message_dict, symbols = asp_try_3(asp_file, instance)

Model: at(2,1,0) at(1,1,1) at(2,1,2) at(3,1,3) at(3,2,4)


In [109]:
symbols

['at(2,1,0)', 'at(1,1,1)', 'at(2,1,2)', 'at(3,1,3)', 'at(3,2,4)']

In [25]:
error_dict

{'AtomUndefined': 0,
 'FileIncluded': 0,
 'GlobalVariable': 0,
 'OperationUndefined': 0,
 'Other': 0,
 'RuntimeError': 4,
 'VariableUnbounded': 0}

In [26]:
messages

{'unsafe': 4}

In [ ]:
try:    
    print("No errors occurred during solving.")
except Exception as e:
    print(e)
    #if control.statistics['solving']['solvers']['conflicts'] > 0:
    print("Errors occurred during solving.")
    # Analyze the conflicts to get more information
    conflicts_analyzed = control.statistics['solving'] #['conflicts_analyzed']
    print("Conflicts analyzed: {}".format(conflicts_analyzed))

In [8]:
aux = '''<block>:2:1-91: error: unsafe variables in:
  move(X,Y,XX,YY):-[#inc_base];#Arith0=1;player(X,Y);#Arith0=(|(X-XX)|+|(Y-YY)|);not minotaur(XX,YY);not wall(XX,YY).
<block>:2:10-12: note: 'XX' is unsafe
<block>:2:13-15: note: 'YY' is unsafe'''

In [9]:
if "unsafe variables" in aux:
    print("Éxito")

Éxito
